In [1]:
import pandas as pd
import os
import sys
import tensorflow as tf
import numpy as np

In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [107]:
feature_names = ["product", "sub_product", "issue", "sub_issue", "state", "zip_code", "company", "company_response", "timely_response", "consumer_disputed"]
cat_features = ["sub_product", "state", "issue", "company"]#, "sub_issue"]
#bucket_features = ['zip_code']
one_hot_features = ['product', 'company_response', 'timely_response', 'zip_code_short']

In [7]:
#df = pd.read_csv('../data/21Feb/26k-consumer-complaints-labels.csv', usecols=feature_names, na_values=0, keep_default_na=False)
df = pd.read_csv('../data/29Feb/cc_large_balanced.csv', usecols=feature_names, na_values=0, keep_default_na=False)

In [18]:
df['zip_code_short'] = df['zip_code'].apply(lambda x: str(x)[0] if len(x)>0 else np.nan)

In [108]:
top_n = 100
top_companies = df['company'].value_counts().head(top_n).index.tolist()
df.loc[~df['company'].isin(top_companies), 'company'] = 'Other'

In [109]:
for feature in cat_features + one_hot_features:
    df[feature] = df[feature].astype("category").cat.codes

In [110]:
one_hot_x = [pd.np.asarray(tf.keras.utils.to_categorical(df[feature_name].values)) for feature_name in one_hot_features]

In [111]:
embedding_x = [pd.np.asarray(df[feature_name].values).reshape(-1) for feature_name in cat_features]#[:-1]

In [101]:
len(one_hot_x)

4

In [112]:
X = one_hot_x + embedding_x

In [43]:
y = np.asarray(df["consumer_disputed"], dtype=np.uint8).reshape(-1)

In [103]:
def wide_and_deep_model(show_summary=True):
    """
    Function defines a Keras model and returns the model as Keras object
    """
    wandb.init(project="consumer-complaints")
    config = wandb.config
    config.name='test_wide_and_deep'
    #config.hidden_layer_size = 256
    config.optimizer = 'adam'
    config.learning_rate = 0.001
    config.data_version = 'cc_large_balanced'
    config.one_hot_features = one_hot_features
    config.cat_features = cat_features

    #cat_features = ["sub_product", "state", "issue", "company", "sub_issue"]
    # one_hot_features = ['product', 'company_response', 'timely_response', 'company', 'issue', 'zip_code_short']

    # one-hot categorical features
    num_products = 11
    num_company_responses = 5
    num_timely_responses = 2
    num_companies = 41
    num_issues = 40
    num_zip_codes = 15

    input_product = tf.keras.Input(shape=(num_products,), name="product_xf")
    input_company_response = tf.keras.Input(shape=(num_company_responses,), name="company_response_xf")
    input_timely_response = tf.keras.Input(shape=(num_timely_responses,), name="timely_response_xf")
    input_zip_code = tf.keras.Input(shape=(num_zip_codes,), name="zip_code_xf")

    # categorical features
    input_sub_product = tf.keras.Input(shape=(1,), name="sub_product_xf")
    input_state = tf.keras.Input(shape=(1,), name="state_xf")
    input_issue = tf.keras.Input(shape=(1,), name="issue_xf")
    input_company = tf.keras.Input(shape=(1,), name="company_xf")
    #input_sub_issue = tf.keras.Input(shape=(1,), name="sub_issue_xf")


    # convert to embeddings
    embed_sub_product = tf.keras.layers.Embedding(70, 35)(input_sub_product)
    embed_sub_product = tf.keras.layers.Reshape((35, ), input_shape=(1, 35))(embed_sub_product)

    embed_state = tf.keras.layers.Embedding(70, 35)(input_state)
    embed_state = tf.keras.layers.Reshape((35, ), input_shape=(1, 35))(embed_state)
    
    embed_issue = tf.keras.layers.Embedding(100, 50)(input_issue)
    embed_issue = tf.keras.layers.Reshape((50, ), input_shape=(1, 50))(embed_issue)
    
    #embed_company = tf.keras.layers.Embedding(110, 50)(input_company)
    #embed_company = tf.keras.layers.Reshape((50, ), input_shape=(1, 50))(embed_company)
    
    #embed_sub_issue = tf.keras.layers.Embedding(70, 35)(input_sub_issue)
    #embed_sub_issue = tf.keras.layers.Reshape((35, ), input_shape=(1, 35))(embed_sub_issue)
    
    # add USE for narrative, and reduce number of dimensions
    
    deep_ff = tf.keras.layers.concatenate(
        [embed_sub_product, embed_state, embed_issue])#, embed_company, embed_sub_issue])
    
    deep = tf.keras.layers.Dense(64, activation='relu')(deep_ff)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)
    #deep = tf.keras.layers.Dense(16, activation='relu')(deep)

    wide = tf.keras.layers.concatenate(
        [input_product, input_company_response, input_timely_response, input_zip_code])

    both = tf.keras.layers.concatenate([deep, wide])

    output = tf.keras.layers.Dense(1, activation='sigmoid')(both) 

    _inputs = [input_product, input_company_response, input_timely_response, input_zip_code, 
               input_sub_product, input_state, 
              input_issue]#, input_company, input_sub_issue] 

    keras_model = tf.keras.models.Model(_inputs, output)
    keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss='binary_crossentropy',  # categorical_crossentropy
                     metrics=[
                         tf.keras.metrics.BinaryAccuracy(),
                         ])
    if show_summary:
        keras_model.summary()

    return keras_model

In [104]:
model = wide_and_deep_model(show_summary=False)

In [105]:
model.fit(x=X, y=y, batch_size=128, validation_split=0.2, epochs=40, 
          callbacks=[WandbCallback(), tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

Train on 96169 samples, validate on 24043 samples
Epoch 1/40
96169/96169 [==============================] - 3s 34us/sample - loss: 0.6854 - binary_accuracy: 0.5532 - val_loss: 0.6762 - val_binary_accuracy: 0.5781
Epoch 2/40
96169/96169 [==============================] - 2s 17us/sample - loss: 0.6754 - binary_accuracy: 0.5775 - val_loss: 0.6758 - val_binary_accuracy: 0.5746
Epoch 3/40
96169/96169 [==============================] - 2s 16us/sample - loss: 0.6735 - binary_accuracy: 0.5782 - val_loss: 0.6736 - val_binary_accuracy: 0.5815
Epoch 4/40
96169/96169 [==============================] - 1s 15us/sample - loss: 0.6724 - binary_accuracy: 0.5796 - val_loss: 0.6745 - val_binary_accuracy: 0.5776
Epoch 5/40
96169/96169 [==============================] - 2s 16us/sample - loss: 0.6718 - binary_accuracy: 0.5809 - val_loss: 0.6737 - val_binary_accuracy: 0.5773
Epoch 6/40
96169/96169 [==============================] - 2s 16us/sample - loss: 0.6710 - binary_accuracy: 0.5822 - val_loss: 0.6748 - 